**Load the cleaned data frames that stored from Assignment 2**

In [3]:
%store -r chefmozaccepts
%store -r chefmozcuisine
%store -r chefmozhours4
%store -r chefmozparking
%store -r geoplaces2
%store -r usercuisine
%store -r userpayment
%store -r userprofile
%store -r rating_final

**Import packages**

In [22]:
import pandas as pd

**3.1. Concatenate all cleaned data frames from ‘Restaurant’ category based on the common column, to create a large data frame and name it as Restaurants.**

In [29]:
Restaurants = geoplaces2.merge(chefmozaccepts, on='placeID', how='left'
                              ).merge(chefmozcuisine, on='placeID', how='left'
                              ).merge(chefmozhours4, on='placeID', how='left'
                              ).merge(chefmozparking, on='placeID', how='left')
Restaurants.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 901 entries, 0 to 900
Data columns (total 23 columns):
placeID           901 non-null int64
latitude          901 non-null float64
longitude         901 non-null float64
the_geom_meter    901 non-null object
name              901 non-null object
address           901 non-null object
city              901 non-null object
state             901 non-null object
country           901 non-null object
alcohol           901 non-null object
smoking_area      901 non-null object
dress_code        901 non-null object
accessibility     901 non-null object
price             901 non-null object
Rambience         901 non-null object
franchise         901 non-null object
area              901 non-null object
other_services    901 non-null object
Rpayment          850 non-null object
Rcuisine          688 non-null object
hours             885 non-null object
days              885 non-null object
parking_lot       901 non-null object
dtypes: float64(2), 

**3.2. Concatenate all cleaned data frames from ‘Customer’ categories based on the common column, to create a large data frame and name it as Customers.**

In [30]:
Customers = userprofile.merge(usercuisine, on='userID', how='left').merge(userpayment, on='userID', how='left')
Customers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 416 entries, 0 to 415
Data columns (total 21 columns):
userID              416 non-null object
latitude            416 non-null float64
longitude           416 non-null float64
smoker              416 non-null object
drink_level         416 non-null object
dress_preference    416 non-null object
ambience            416 non-null object
transport           416 non-null object
marital_status      416 non-null object
hijos               416 non-null object
birth_year          416 non-null int64
interest            416 non-null object
personality         416 non-null object
religion            416 non-null object
activity            416 non-null object
color               416 non-null object
weight              416 non-null int64
budget              416 non-null object
height              416 non-null float64
Rcuisine            416 non-null object
Upayment            411 non-null object
dtypes: float64(3), int64(2), object(16)
memory usage:

**3.3. Import the required package and create an engine for the sqlalchemy application:**

In [31]:
import sqlalchemy as sqla

In [32]:
# Create connection to a database
db = sqla.create_engine('sqlite:///restaurant_customer.db', echo=False)

**3.4. Create three SQL tables from three data frames:**

In [40]:
# Store data frames into tables
try:
    Restaurants.to_sql('Restaurants', db)
    Customers.to_sql('Customers', db)
    rating_final.to_sql('Ratings', db)
except ValueError:
    print('Table exist')

**3.5. Answer all the questions from the previous assignment (Assignment 2), using SQL queries; except for Question 2.2.8 which is not needed to be answered in Assignment 3.**

2.1.3. What are the names of different restaurants in the state of ‘tamaulipas’?

In [41]:
pd.read_sql('select distinct(name) from Restaurants where state = "Tamaulipas"', db)

,name
0,Gorditas Dona Tota
1,churchs
2,tortas hawai
3,tacos de la estacion
4,Carnitas Mata Calle 16 de Septiembre
5,TACOS CORRECAMINOS
6,la perica hamburguesa
7,TACOS EL GUERO
8,tacos de barbacoa enfrente del Tec
9,Carreton de Flautas y Migadas


2.1.4. How many different customers used public transport for going to the restaurants?

In [44]:
pd.read_sql('select count(distinct(userID)) as result from Customers where transport = "public"', db)

,result
0,89


2.1.5. What is the least popular payment method among customers?

In [136]:
pd.read_sql('select Upayment from Customers where Upayment <> "None"' +
            'group by Upayment order by count(Upayment) limit 1', db)

,Upayment
0,American_Express


2.1.6. How many (different) restaurants work until 19:00 in the evenings?

In [88]:
pd.read_sql('select count(distinct(placeID)) as result from Restaurants where hours like "%-19:00;%"', db)

,result
0,8


Note: the result is different from Assignment 2 probably because some placeID in chefmozhours4 are not in geoplaces2 and therefore not in the merged data frame Restaurants

2.1.7. Which type of food (rcuisine) is served most among the restaurants?

In [116]:
pd.read_sql('select Rcuisine from Restaurants where Rcuisine <> "None"' +
            'group by Rcuisine order by count(Rcuisine) desc limit 1', db)

,Rcuisine
0,Mexican


2.1.8. What is the percentage of customers who were born between 1980 and 1990?

In [127]:
pd.read_sql('select count(distinct(userID))*100/(select count(distinct(userID)) from Customers) ' + 
            'as "percentage(%)" from Customers where birth_year between 1980 and 1990', db)

,percentage(%)
0,63


2.1.9. What is the percentage of students with a medium budget preferring walking to the restaurants?

In [120]:
pd.read_sql('select count(distinct(userID))*100/(select count(distinct(userID)) from Customers) as "percentage(%)"' + 
            'from Customers where activity = "student" and budget = "medium" and transport = "on foot"', db)

,percentage(%)
0,5


2.2.1. What are the names of restaurants that do not have public parking lots?

In [108]:
pd.read_sql('select distinct(placeID), name from Restaurants where placeID not in ' +
            '(select distinct(placeID) from Restaurants where parking_lot = "public")', db)

,placeID,name
0,135018,El Oceano Dorado
1,135016,Arrachela Grill
2,132773,El cotorreo
3,135000,Restaurant los Pinos
4,135019,Restaurant Bar Coty y Pablo
5,132767,Restaurant Familiar El Chino
6,132766,Mikasa
7,135011,Los Vikingos
8,134999,Kiku Cuernavaca
9,134976,Log Yin


2.2.2. What are the addresses of restaurants which only accept ‘cash’?

In [109]:
pd.read_sql('select distinct(placeID), address from Restaurants ' + 
            'group by placeID having count(distinct(Rpayment)) = 1 and Rpayment = "cash"', db)

,placeID,address
0,132560,frente al tecnologico
1,132572,De Guadalupe 460 San Miguelito
2,132584,frente al tecnologico
3,132594,16 de Septiembre
4,132608,cd. mier
5,132609,tampico
6,132613,lic. Emilio portes gil
7,132667,calle emilio portes gil
8,132715,Zaragoza entre Francisco Zarco y Lopez Velarde
9,132717,Zaragoza entre Francisco Zarco y Lopez Velarde


2.2.3. Name the cities where the restaurants cook and serve ‘fast food’?

In [111]:
pd.read_sql('select distinct(city) from Restaurants where Rcuisine = "Fast_Food"', db)

,city
0,Cuernavaca
1,San Luis Potosi
2,Soledad
3,Ciudad Victoria


2.2.4. What is the most common ‘rating’ among customers with family?

In [144]:
pd.read_sql('select r.rating as rating from ' + 
            '(select distinct(userID), ambience from Customers) as c ' + 
            'inner join Ratings r on c.userID = r.userID ' + 
            'where c.ambience = "family" group by r.rating order by count(r.userID) desc limit 1', db)

,rating
0,2


2.2.5. What types of food (‘rcuisine’) received the highest rank from people with ‘low’ budget?

In [150]:
pd.read_sql('select distinct(r.Rcuisine) from Ratings ra ' + 
            'inner join Restaurants r on ra.placeID = r.placeID ' +
            'inner join Customers c on ra.userID = c.userID ' +
            'where c.budget = "low" and ra.food_rating = 2 and r.Rcuisine <> "None"', db)

,Rcuisine
0,Mexican
1,Vietnamese
2,Pizzeria
3,Cafeteria
4,Contemporary
5,Fast_Food
6,Seafood
7,Bar
8,Bar_Pub_Brewery
9,International


2.2.6. What is the average of ‘service rating’, received from ‘social drinkers’ about restaurants which just served ‘Wine-Beer’?

In [160]:
pd.read_sql('select avg(ra.service_rating) as average_service_rating from Ratings ra ' + 
            'inner join (select distinct(placeID), alcohol from Restaurants) as r on ra.placeID = r.placeID ' +
            'inner join (select distinct(userID), drink_level from Customers) as c on ra.userID = c.userID ' +
            'where c.drink_level = "social drinker" and r.alcohol = "Wine-Beer"', db)

,average_service_rating
0,1.386364


2.2.7. How many smokers gave zero ‘service rating’ to the restaurants without an open area?

In [166]:
pd.read_sql('select count(distinct(ra.userID)) as number_of_smokers from Ratings ra ' + 
            'inner join (select distinct(placeID), area from Restaurants) as r on ra.placeID = r.placeID ' +
            'inner join (select distinct(userID), smoker from Customers) as c on ra.userID = c.userID ' +
            'where c.smoker = "true" and r.area <> "open" and ra.service_rating = 0', db)

,number_of_smokers
0,15
